In [ ]:
# 03_portfolio_optimization.ipynb

"""
This Jupyter Notebook demonstrates a simple Markowitz optimization approach
using random portfolio generation for TSLA, BND, and SPY.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Load Data
tsla = pd.read_csv('../data/raw/TSLA.csv', parse_dates=['Date'], index_col='Date').sort_index()
bnd = pd.read_csv('../data/raw/BND.csv', parse_dates=['Date'], index_col='Date').sort_index()
spy = pd.read_csv('../data/raw/SPY.csv', parse_dates=['Date'], index_col='Date').sort_index()



In [ ]:
# 2. Compute Returns
df = pd.DataFrame({
    'TSLA': tsla['Close'].pct_change(),
    'BND': bnd['Close'].pct_change(),
    'SPY': spy['Close'].pct_change()
}).dropna()



In [ ]:
# 3. Random Portfolio Simulation
num_portfolios = 5000
risk_free_rate = 0.02
results = []



In [ ]:
for _ in range(num_portfolios):
    weights = np.random.random(3)
    weights /= np.sum(weights)
    # Annualized Return
    port_return = np.sum(df.mean() * weights) * 252
    # Annualized Volatility
    port_vol = np.sqrt(np.dot(weights.T, np.dot(df.cov() * 252, weights)))
    # Sharpe Ratio
    sharpe = (port_return - risk_free_rate) / port_vol
    
    results.append({
        'weights': weights,
        'return': port_return,
        'volatility': port_vol,
        'sharpe': sharpe
    })

results_df = pd.DataFrame(results)
best_portfolio = results_df.loc[results_df['sharpe'].idxmax()]

In [ ]:
# 4. Print Best Portfolio
print("Optimal Weights:")
tickers = ['TSLA', 'BND', 'SPY']
for t, w in zip(tickers, best_portfolio['weights']):
    print(f"{t}: {w:.2%}")

print(f"Expected Annual Return: {best_portfolio['return']:.2%}")
print(f"Volatility: {best_portfolio['volatility']:.2%}")
print(f"Sharpe Ratio: {best_portfolio['sharpe']:.2f}")




In [ ]:
# 5. Visualization (Optional)
plt.figure(figsize=(10, 6))
plt.scatter(results_df['volatility'], results_df['return'], c=results_df['sharpe'], cmap='viridis')
plt.colorbar(label='Sharpe Ratio')
plt.scatter(best_portfolio['volatility'], best_portfolio['return'], c='red', s=50, label='Max Sharpe')
plt.xlabel('Volatility')
plt.ylabel('Annual Return')
plt.title('Random Portfolio Simulation')
plt.legend()
plt.show()